## Luftdata med kartor

Simon Winter, Infontology http://infontology.org



I den här lektionen ska vi se hur man kan hitta data för luftföroreningar från det nätverk av sammankopplade sensorer som finns på Luftdata.se

In [1]:
import requests
import pandas as pd
import gmaps
from credentials import key

/Users/management/.local/lib/python3.7/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# https://console.developers.google.com/apis/dashboard?project=natural-furnace-264420
gmaps.configure(api_key=key)

In [3]:
latitude = '55.611437'
longitude = '12.994264'
distance = '150' # Avstånd i kilometer

Med hjälp av koordinaterna så skapar vi urlen. Lämna tomma platser inom {} i textsträngen, och ge variabelnamnen i format-parentesen

In [4]:
url = 'http://api.luftdaten.info/v1/filter/area={},{},{}'.format(latitude, longitude, distance)
url ## Om man ger ett variabelnamn på sista raden av en cell skrivs värdet ut. Det går också att göra print(url)

'http://api.luftdaten.info/v1/filter/area=55.611437,12.994264,150'

Requests-biblioteket gör det lätt att hantera frågor till servern. 

In [5]:
response = requests.get(url)

In [6]:
sensors = response.json()
number_of_sensors = len(sensors)
number_of_sensors

143

Om number_of_sensors är större än 0 så finns det sensorer i listan. Då kan man få fram sensorvärdena på olika sätt. Det första värdet ligger alltid i sensors[0]

In [7]:
sens2 = pd.DataFrame(columns=['latitude', 'longitude'])

In [8]:
sens2

,latitude,longitude


In [9]:
def create_sensordict (sensor):
    sensordict = {}
    sensordict['longitude'] = sensor['location']['longitude']
    sensordict['latitude'] = sensor['location']['latitude']
    for stype in sensor['sensordatavalues']:
        sensordict[stype['value_type']] = stype['value']
    return sensordict

In [10]:
for sensor in sensors:
    sens2 = sens2.append (create_sensordict(sensor), ignore_index=True)

    

In [11]:
sens2

,latitude,longitude,humidity,temperature,P1,P2,pressure,pressure_at_sealevel
0,55.796,13.09,99.90,4.60,NaN,NaN,NaN,NaN
1,55.796,13.09,NaN,NaN,4.83,1.53,NaN,NaN
2,55.61,12.1,0.00,7.19,NaN,NaN,100132.84,100716.17
3,55.61,12.1,NaN,NaN,10.07,7.45,NaN,NaN
4,55.658,13.358,100.00,6.66,NaN,NaN,99111.06,99903.67
...,...,...,...,...,...,...,...,...
138,56.144,13.394,NaN,NaN,5.20,2.03,NaN,NaN
139,55.606,13.024,81.65,8.94,NaN,NaN,100643.66,100743.63
140,55.606,13.024,NaN,NaN,10.27,6.07,NaN,NaN
141,55.61,12.1,0.00,7.08,NaN,NaN,100144.88,100728.51


In [12]:
sens2 = sens2.astype('float')

In [13]:
sens2.dtypes

latitude                float64
longitude               float64
humidity                float64
temperature             float64
P1                      float64
P2                      float64
pressure                float64
pressure_at_sealevel    float64
dtype: object

In [14]:
humid_values = sens2[sens2['humidity'].notnull()]

In [15]:
locations = humid_values[['latitude', 'longitude']]
weights = humid_values['humidity']

In [16]:
locations

,latitude,longitude
0,55.796000,13.090000
2,55.610000,12.100000
4,55.658000,13.358000
6,55.648000,13.208000
8,55.674000,13.354000
...,...,...
132,55.713015,13.336429
134,55.654000,12.138000
137,56.144000,13.394000
139,55.606000,13.024000


In [17]:
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=weights)
fig.add_layer(heatmap_layer)
fig



Figure(layout=FigureLayout(height='420px'))

In [6]:
sensors[0]

{'sampling_rate': None,
 'location': {'country': 'SE',
  'altitude': '8.2',
  'id': 8180,
  'indoor': 0,
  'exact_location': 0,
  'longitude': '13.024',
  'latitude': '55.606'},
 'sensordatavalues': [{'value_type': 'temperature',
   'id': 12788794817,
   'value': '7.86'},
  {'value_type': 'pressure', 'id': 12788794818, 'value': '101336.69'},
  {'value_type': 'humidity', 'id': 12788794820, 'value': '78.45'},
  {'value_type': 'pressure_at_sealevel', 'value': 101437.74}],
 'id': 6019977378,
 'timestamp': '2020-01-09 10:48:53',
 'sensor': {'sensor_type': {'name': 'BME280',
   'id': 17,
   'manufacturer': 'Bosch'},
  'id': 16150,
  'pin': '11'}}

Det finns olika typer av värden:

* P1 är PM10, alltså partiklar mindre än 10 mikrometer
* P2 är PM2.5, alltså partiklar mindre än 2,5 mikrometer
* temperature är temperatur
* humidity är luftfuktighet
* pressure är lufttryck